## Author: Akash

### Creating Reweighted Redlining Index and merging with ACS Census Tracts

Data Source: Redlining and Census Tract Intersection files for each city extracted using the spatial outlining step

In [ ]:
# Importing needed libraries
import pandas as pd
import geopandas as gpd

In [ ]:
#Load the spatial intersection file generated using spatial outlining notebook
intersection = pd.read_csv('/home/jovyan/work/COVIDRedlining/data/boston/boston_redline_intersection.csv')
intersection.head()

,GEOID,polygon_id,weight
0,25025010600,4947.0,0.095512
1,25025010600,4918.0,0.904488
2,25025010204,4947.0,1.000000
3,25025010404,4947.0,0.998484
4,25025010404,4919.0,0.001516


In [ ]:
#Load the Redlining Shapefile
redline = gpd.read_file('/home/jovyan/work/COVIDRedlining/data/boston/boston redlining/boston_redlining.shp')
redline.head()

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry
0,9543.0,MA,Boston,None,A1,A,"{ ""32"" : """", ""3k"" : ""new construction "", ""3q""...","POLYGON ((-71.11985 42.32301, -71.11990 42.321..."
1,5015.0,MA,Boston,None,B1,B,"{ ""1d"" : ""98%"", ""3i"" : ""$6,500-$7,500 1933-36 ...","POLYGON ((-71.17193 42.34689, -71.17222 42.346..."
2,9545.0,MA,Boston,None,B2,B,"{ ""1a"" : ""high - hilly"", ""3j"" : "" $8,000-$12,0...","POLYGON ((-71.14608 42.34329, -71.14567 42.342..."
3,4949.0,MA,Boston,None,B3,B,"{ ""2a"" : ""business and professional men"", ""2b""...","POLYGON ((-71.07162 42.35945, -71.07215 42.357..."
4,4946.0,MA,Boston,None,B4,B,"{ ""3d"" : ""good good"", ""2c"" : ""0% "", ""1d"" : ""9...","POLYGON ((-71.11527 42.30196, -71.11761 42.302..."


In [ ]:
#Visualise value counts for each HOLC grade obtained from the Redlining Shapefile
redline['holc_grade'].value_counts()

C    18
D    12
B     8
A     1
Name: holc_grade, dtype: int64

In [ ]:
#Convert HOLC Grade to Ordinal Values
def numerical_redlining_value(row):
    if row['holc_grade'] == 'A':
      val = 1
    elif row['holc_grade'] == 'B':
      val = 2
    elif row['holc_grade'] == 'C':
      val = 3
    elif row['holc_grade'] == 'D':
      val = 4
    elif row['holc_grade'] == 'E':
      val = 5
    else:
      val = 0

    return val

#Assign ordinal values to a new column
redline['numerical_grade'] = redline.apply(numerical_redlining_value, axis=1)

In [ ]:
#Merge intersection shapefile with redline shapefile
output_combined=pd.merge(redline, intersection, how='left', on='polygon_id')
output_combined.head()

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry,numerical_grade,GEOID,weight
0,9543.0,MA,Boston,None,A1,A,"{ ""32"" : """", ""3k"" : ""new construction "", ""3q""...","POLYGON ((-71.11985 42.32301, -71.11990 42.321...",1,25025981800,0.001120
1,9543.0,MA,Boston,None,A1,A,"{ ""32"" : """", ""3k"" : ""new construction "", ""3q""...","POLYGON ((-71.11985 42.32301, -71.11990 42.321...",1,25025981000,0.102719
2,9543.0,MA,Boston,None,A1,A,"{ ""32"" : """", ""3k"" : ""new construction "", ""3q""...","POLYGON ((-71.11985 42.32301, -71.11990 42.321...",1,25025110607,0.000334
3,9543.0,MA,Boston,None,A1,A,"{ ""32"" : """", ""3k"" : ""new construction "", ""3q""...","POLYGON ((-71.11985 42.32301, -71.11990 42.321...",1,25025110601,0.356997
4,9543.0,MA,Boston,None,A1,A,"{ ""32"" : """", ""3k"" : ""new construction "", ""3q""...","POLYGON ((-71.11985 42.32301, -71.11990 42.321...",1,25025120105,1.000000


In [ ]:
#Viewing the shape of the data
output_combined.shape

(379, 11)

In [ ]:
#Check if rows are NA - some polygons have NULL IDs 
output_combined[output_combined['polygon_id'].isna()]

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry,numerical_grade,GEOID,weight


In [ ]:
#Removing polygon_ids with NA/NULL values (Check if polygons have corresponding GEOIDs before deleteing)
output_combined = output_combined[output_combined['polygon_id'].notna()]
output_combined.shape

(379, 11)

In [ ]:
#Subsetting redline numerical index and spatial weights
redline_index = output_combined[['GEOID','polygon_id','numerical_grade','weight']]
redline_index[redline_index['GEOID']==25025010600]

,GEOID,polygon_id,numerical_grade,weight
190,25025010600,4947.0,3,0.095512
323,25025010600,4918.0,4,0.904488


In [ ]:
#Reweighting the redlining numerical index
redline_index['reweighted_redline_index'] = redline_index['numerical_grade'] * redline_index['weight']
redline_index[redline_index['GEOID']==25025010600]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GEOID,polygon_id,numerical_grade,weight,reweighted_redline_index
190,25025010600,4947.0,3,0.095512,0.286535
323,25025010600,4918.0,4,0.904488,3.617954


In [ ]:
#Grouping reweighted redline index by GEOID
redline_index_grouped = redline_index[['GEOID','reweighted_redline_index']].groupby('GEOID').sum()
redline_index_grouped[redline_index_grouped.index==25025010600]

,reweighted_redline_index
GEOID,
25025010600,3.904488


In [ ]:
#Visualising the dataset
redline_index_grouped.head()

,reweighted_redline_index
GEOID,
25017373100,2.0
25017373500,2.0
25021400100,3.0
25021400200,3.0
25021400300,3.0


In [ ]:
#Reconverting reweighted values to ordinal
def ordinal_values(row):
    if row['reweighted_redline_index'] > 0 and row['reweighted_redline_index'] <= 1:
      val = 1
    elif row['reweighted_redline_index'] > 1 and row['reweighted_redline_index'] <= 2:
      val = 2
    elif row['reweighted_redline_index'] > 2 and row['reweighted_redline_index'] <= 3:
      val = 3
    elif row['reweighted_redline_index'] > 3 and row['reweighted_redline_index'] <= 4.1:
      val = 4
    elif row['reweighted_redline_index'] > 4.1 and row['reweighted_redline_index'] <= 5:
      val = 5
    else:
      val = 0

    return val

redline_index_grouped['ordinal_value'] = redline_index_grouped.apply(ordinal_values, axis=1)

In [ ]:
redline_index_grouped.head()

,reweighted_redline_index,ordinal_value
GEOID,,
25017373100,2.0,2
25017373500,2.0,2
25021400100,3.0,3
25021400200,3.0,3
25021400300,3.0,3


In [ ]:
#Load Census Tract Data
ct_shape = gpd.read_file('/home/jovyan/work/COVIDRedlining/data/boston/boston census tracts/tl_2019_25_tract.shp')
ct_shape.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,25,027,724100,25027724100,7241,Census Tract 7241,G5020,S,53031300,1639215,+42.2566908,-072.1581690,"POLYGON ((-72.21782 42.27018, -72.21765 42.270..."
1,25,027,759100,25027759100,7591,Census Tract 7591,G5020,S,25548737,1449315,+42.2096822,-072.0401777,"POLYGON ((-72.07888 42.21475, -72.07880 42.214..."
2,25,001,012601,25001012601,126.01,Census Tract 126.01,G5020,S,3467387,9794,+41.6624989,-070.3180404,"POLYGON ((-70.34011 41.65738, -70.33993 41.657..."
3,25,025,170501,25025170501,1705.01,Census Tract 1705.01,G5020,S,955798,593136,+42.4150328,-070.9902217,"POLYGON ((-71.00017 42.40962, -70.99989 42.410..."
4,25,027,709701,25027709701,7097.01,Census Tract 7097.01,G5020,S,4528209,138136,+42.5450353,-071.7748626,"POLYGON ((-71.79318 42.55169, -71.79318 42.551..."


In [ ]:
#Converting Census Tract Object Datatype to Float
ct_shape['GEOID'] = ct_shape['GEOID'].astype(float)
#Merging redline index scores with Census Tract data
ct_shape = ct_shape.merge(redline_index_grouped,how='inner',on='GEOID')
ct_shape.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,reweighted_redline_index,ordinal_value
0,25,025,010600,2.502501e+10,106,Census Tract 106,G5020,S,339678,0,+42.3476422,-071.0793745,"POLYGON ((-71.08560 42.34788, -71.08531 42.347...",3.904488,4
1,25,025,120500,2.502512e+10,1205,Census Tract 1205,G5020,S,213706,0,+42.3206587,-071.1042025,"POLYGON ((-71.10845 42.32239, -71.10798 42.322...",3.725805,4
2,25,025,070600,2.502507e+10,706,Census Tract 706,G5020,S,133036,0,+42.3432581,-071.0753887,"POLYGON ((-71.07921 42.34270, -71.07856 42.343...",4.000000,4
3,25,025,140201,2.502514e+10,1402.01,Census Tract 1402.01,G5020,S,1322336,99423,+42.2375297,-071.1305801,"POLYGON ((-71.13924 42.23366, -71.13841 42.234...",3.000000,3
4,25,025,082100,2.502508e+10,821,Census Tract 821,G5020,S,370577,0,+42.3094485,-071.0872292,"POLYGON ((-71.09208 42.30995, -71.09182 42.310...",3.000000,3


In [ ]:
#Saving file as geojson
ct_shape.to_file('boston_redlined_numerical_values.geojson',driver='GeoJSON')